## Aula de API

### Bibliotecas Utilizadas

- Flask
- FastAPI
- uvicorn

#### Pastas:
```md
meu_pacote/
├── codigos
│   ├── __init__.py
│   └── codigo.py
├── LICENSE
├── README.md
├── setup.py
└── tests
    └── test_codigo.py
```
#### Comando para criação do pacote:
```bash
python setup.py sdist bdist_wheel
```

#### Upload para o pypi:
##### Configurar o arquivo $home/.pypirc

```bash
[pypi]
  username = __token__
  password = {token_do_pypi}

```
##### Rodar o comando:
```bash
twine upload dist/*
```

In [1]:
# app.py

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///example.db'
db = SQLAlchemy(app)

class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)

In [ ]:
# app.py, blueprint

from flask import Blueprint, render_template

auth_bp = Blueprint('auth', __name__)

@auth_bp.route('/login')
def login():
    return render_template('login.html')

# Integrando o blueprint no aplicativo principal
app.register_blueprint(auth_bp, url_prefix='/auth')

In [ ]:
# main.py

from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class Item(BaseModel):
    name: str
    description: str = None
    price: float
    tax: float = None

In [ ]:
# main.py

from fastapi import Depends, FastAPI, HTTPException
from fastapi.security import OAuth2PasswordBearer

app = FastAPI()

oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

# Uso de dependência avançada
def get_current_user(token: str = Depends(oauth2_scheme)):
    credentials_exception = HTTPException(
        status_code=401,
        detail="Could not validate credentials",
        headers={"WWW-Authenticate": "Bearer"}
    )
    # Lógica para validar o token e obter o usuário atual
    return current_user

@app.get("/users/na")
async def read_user_na(current_user: User = Depends(get_current_user)):
    return {"username": current_user.username}